In [0]:
%run "../includes/configuration"

In [0]:
%run
"../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
#movies_df = spark.sql("select * from movie_silver.movies").filter(f"file_date = '{v_file_date}'")
movies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies") \
                    .filter(f"file_date = '{v_file_date}'")


In [0]:
#genres_df = spark.sql("select * from movie_silver.genres").filter(f"file_date = '{v_file_date}'")
genres_df = spark.read.format("delta").load(f"{silver_folder_path}/genres")

In [0]:
#movies_genres_df = spark.sql("select * from movie_silver.movie_genre").filter(f"file_date = '{v_file_date}'")
movies_genres_df = spark.read.format("delta").load(f"{silver_folder_path}/movie_genre") \
                    .filter(f"file_date = '{v_file_date}'")


In [0]:
genres_mov_gen_df = genres_df.join(movies_genres_df, genres_df.genre_id == movies_genres_df.genre_id,
                                    "inner") \
                                .select(genres_df.genre_id,genres_df.genre_name,movies_genres_df.movie_id)




In [0]:
movies_filter_df = movies_df.filter(movies_df.year_release_date >= "2000")

In [0]:
display(movies_filter_df)

In [0]:
results_movies_genres_df = movies_filter_df.join(genres_mov_gen_df, movies_filter_df.movie_id == genres_mov_gen_df.movie_id, "inner") 
                                

In [0]:
results_df = results_movies_genres_df.select(movies_filter_df.movie_id,results_movies_genres_df.genre_id,"year_release_date","genre_name","budget","revenue")

In [0]:
from pyspark.sql.functions import current_timestamp, desc, dense_rank, asc, sum, col, lit
from pyspark.sql.window import Window

results_group_by_df = results_df.groupBy("movie_id","genre_id","year_release_date","genre_name").agg(sum("budget").alias("total_budget"),sum("revenue").alias("total_revenue"))
result_dense_rank_df = Window.partitionBy("year_release_date").orderBy(desc("total_budget"),
                                                                       desc("total_revenue"))
                                                                
final_df = results_group_by_df.withColumn("dense_rank",dense_rank().over(result_dense_rank_df)) \
                              .withColumn("created_date", lit(v_file_date))

#### Escribir datos en datalake en formato "Parquet"

In [0]:
#overwrite_partition(final_df, "movie_gold", "results_group_movie_genre", "created_date") 

In [0]:
#final_df.write.mode("overwrite").partitionBy("created_date").format("parquet").saveAsTable("movie_gold.results_group_movie_genre")

#### Escribir en Delta

In [0]:
merge_key = 'tgt.genre_id = src.genre_id and tgt.movie_id = src.movie_id '
merge_delta_data(final_df,"/mnt/moviehistory7809/gold","movie_gold", "results_group_movie_genre", "created_date", merge_key)


In [0]:
%sql
select created_date,count(*) 
from movie_gold.results_group_movie_genre
group by created_date